In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

KeyboardInterrupt: 

In [ ]:
df = pd.read_parquet(r"data/joind_output.parquet", engine="pyarrow")

In [ ]:
df

,neid,LOCALCELLID,dateday,DLEARFCN,cellreselpriority_x,qrxlevmin_x,snonintrasearch,thrshservlow,cellreselpriority_y,qoffsetfreq,...,TIM_PRB_UTIL_MEAN_UL_DEN (Unid),TIM_PRB_UTIL_MEAN_UL_NUM (Unid),TIM_THROU_USER_PDCP_DL (Kbps),TIM_THROU_USER_PDCP_DL_DEN (s),TIM_THROU_USER_PDCP_DL_NUM (Kb),TIM_THROU_USER_PDCP_UL (Kbps),TIM_THROU_USER_PDCP_UL_DEN (s),TIM_THROU_USER_PDCP_UL_NUM (Kb),TIM_USERS_RRC_CONN_MEAN_SUM (Unid),TIM_VOLUME_DADOS_DL_ALLOP (KB)
0,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,...,50.0,949.0,9455.377060,564960.0,5.341910e+09,839.193198,426603.0,358002336.0,6.847,680549.164062
1,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,...,50.0,871.1,12700.061449,178750.0,2.270136e+09,627.378004,282349.0,177139552.0,6.024,303761.855469
2,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,...,50.0,788.8,10109.848984,65993.0,6.671793e+08,173.679601,336718.0,58481048.0,4.944,86050.669922
3,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,...,50.0,690.4,16529.563141,14238.0,2.353479e+08,314.652147,255292.0,80328176.0,3.606,32611.041016
4,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,...,50.0,799.5,9299.013731,109313.0,1.016503e+09,364.658493,547468.0,199638856.0,5.150,130931.255859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3905365,SLRJZ602,9,2023-10-01,9410,7,-60,15,1,5,dB12,...,75.0,466.2,63681.882353,17.0,1.082592e+06,1684.788396,2344.0,3949144.0,1.003,942.801758
3905366,SLRJZ602,9,2023-10-01,9410,7,-60,15,1,5,dB12,...,75.0,466.2,3600.000000,1.0,3.600000e+03,1677.964082,2283.0,3830792.0,1.017,724.338867
3905367,SLRJZ602,9,2023-10-01,9410,7,-60,15,1,5,dB12,...,75.0,465.5,26472.000000,6.0,1.588320e+05,2215.947977,1384.0,3066872.0,1.000,602.082031
3905368,SLRJZ602,9,2023-10-01,9410,7,-60,15,1,5,dB12,...,75.0,466.1,3960.000000,2.0,7.920000e+03,1607.312633,2351.0,3778792.0,1.006,782.392578


In [ ]:
na_counts = df.isna().sum()
na_counts_df = na_counts.reset_index()
na_counts_df.columns = ['Coluna', 'Qtd_NaNs']
print(na_counts_df)

                                Coluna  Qtd_NaNs
0                                 neid         0
1                          LOCALCELLID         0
2                              dateday         0
3                             DLEARFCN         0
4                  cellreselpriority_x         0
..                                 ...       ...
56       TIM_THROU_USER_PDCP_UL (Kbps)    311579
57      TIM_THROU_USER_PDCP_UL_DEN (s)    234288
58     TIM_THROU_USER_PDCP_UL_NUM (Kb)    234288
59  TIM_USERS_RRC_CONN_MEAN_SUM (Unid)    234288
60      TIM_VOLUME_DADOS_DL_ALLOP (KB)    234288

[61 rows x 2 columns]


In [ ]:
df = df.drop(columns=["TIM_BUFFER_MEAN_USER_DL_VOLTE (Unid)", "TIM_IRAT_HO_SUCC_OUT (%)"])
df = df.dropna(axis=0)


In [ ]:
df["qoffsetfreq"] = df["qoffsetfreq"].str.replace("dB", "", regex=False).astype(float)

In [ ]:
df = pd.get_dummies(df, columns=["DLEARFCN"])

In [ ]:
num_cols = df.select_dtypes(include="number").columns
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
df = df.astype({col: int for col in df.select_dtypes('bool').columns})

In [ ]:
df

,neid,LOCALCELLID,dateday,cellreselpriority_x,qrxlevmin_x,snonintrasearch,thrshservlow,cellreselpriority_y,qoffsetfreq,qrxlevmin_y,...,DLEARFCN_1550,DLEARFCN_2800,DLEARFCN_3075,DLEARFCN_3100,DLEARFCN_3150,DLEARFCN_3200,DLEARFCN_3350,DLEARFCN_3375,DLEARFCN_9410,DLEARFCN_9510
48168,18NLITAG01,-0.630285,2023-09-18,0.743822,0.0,0.0,-0.516712,0.749372,1.194069,0.192445,...,0,0,0,0,0,0,0,0,0,0
48169,18NLITAG01,-0.630285,2023-09-18,0.743822,0.0,0.0,-0.516712,0.749372,1.194069,0.192445,...,0,0,0,0,0,0,0,0,0,0
48170,18NLITAG01,-0.630285,2023-09-18,0.743822,0.0,0.0,-0.516712,0.749372,1.194069,0.192445,...,0,0,0,0,0,0,0,0,0,0
48171,18NLITAG01,-0.630285,2023-09-18,0.743822,0.0,0.0,-0.516712,0.749372,1.194069,0.192445,...,0,0,0,0,0,0,0,0,0,0
48172,18NLITAG01,-0.630285,2023-09-18,0.743822,0.0,0.0,-0.516712,0.749372,1.194069,0.192445,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3905239,SLRJZ602,-0.449366,2023-09-28,0.743822,0.0,0.0,-0.516712,-1.800133,1.046723,0.192445,...,0,0,0,0,0,0,0,0,1,0
3905240,SLRJZ602,-0.449366,2023-09-28,0.743822,0.0,0.0,-0.516712,-1.800133,1.046723,0.192445,...,0,0,0,0,0,0,0,0,1,0
3905241,SLRJZ602,-0.449366,2023-09-28,0.743822,0.0,0.0,-0.516712,-1.800133,1.046723,0.192445,...,0,0,0,0,0,0,0,0,1,0
3905243,SLRJZ602,-0.449366,2023-09-28,0.743822,0.0,0.0,-0.516712,-1.800133,1.046723,0.192445,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
df.to_parquet("data/pp_joind.parquet", engine="pyarrow")